In [2]:
import pandas as pd
import scipy.stats as stats
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import string



In [3]:
df = pd.read_csv('world-happiness-report-2021.csv')
#df.head()
#df.info()
#df.describe()

df2 = pd.read_csv('world-happiness-report.csv')
df2.head()

#Création liste des régions du monde par pays
country_region = df.iloc[:,0:2]

#Mise en forme des 2 datasets en prévision du concat
#dataset de 2021 :
df_1 = df.iloc[:,:12]
df_1 = df_1.drop(df.iloc[:,3:6], axis=1)
df_1 = df_1.drop('Regional indicator', axis=1)
df_1['year']='2021'

#dataset historique depuis 2005 :
df_2 = df2.iloc[:, :-2]
dico = {'Life Ladder' : 'Ladder score', 'Log GDP per capita' : 'Logged GDP per capita', 'Healthy life expectancy at birth' : 'Healthy life expectancy'}
df_2.rename(dico, axis=1, inplace=True)

#création du fichier contenants les données depuis 2005 :
df_full = pd.concat([df_1, df_2], axis=0)
df_full['year'] = df_full['year'].astype('int')
df_full = df_full.sort_values(by='Country name', ascending=True)

#ajout colonne Region d'après le df country_region créé précédemment :
df_full = df_full.merge(country_region, on = 'Country name', how ='inner')

#ajout du rank par année de chaque pays
df_full["rank"] = df_full.groupby("year")["Ladder score"].rank("dense", ascending=False).astype('int')


#Remplacement des NaN par les moyennes des variables en fonction de chaque pays
for i in df_full.iloc[:,2:8].columns : 
    df_full[i] = df_full[i].fillna(df_full.groupby('Country name')[i].transform("mean"))
    

df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2035 entries, 0 to 2034
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Country name                  2035 non-null   object 
 1   Ladder score                  2035 non-null   float64
 2   Logged GDP per capita         2035 non-null   float64
 3   Social support                2035 non-null   float64
 4   Healthy life expectancy       2035 non-null   float64
 5   Freedom to make life choices  2035 non-null   float64
 6   Generosity                    2035 non-null   float64
 7   Perceptions of corruption     2035 non-null   float64
 8   year                          2035 non-null   int32  
 9   Regional indicator            2035 non-null   object 
 10  rank                          2035 non-null   int32  
dtypes: float64(7), int32(2), object(2)
memory usage: 174.9+ KB


In [4]:
df_full.head()

,Country name,Ladder score,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,year,Regional indicator,rank
0,Afghanistan,4.402,7.540,0.552,51.20,0.679,0.190,0.850,2009,South Asia,91
1,Afghanistan,3.131,7.718,0.526,52.88,0.509,0.104,0.871,2014,South Asia,135
2,Afghanistan,3.983,7.702,0.529,53.20,0.389,0.080,0.881,2015,South Asia,121
3,Afghanistan,4.220,7.697,0.559,53.00,0.523,0.042,0.793,2016,South Asia,114
4,Afghanistan,2.662,7.697,0.491,52.80,0.427,-0.121,0.954,2017,South Asia,142


In [29]:

# Sélection du pays cherché :

Pays_select = 'Italy'

page_pays= urlopen("https://en.wikipedia.org/wiki/"+Pays_select)


In [30]:
soup = BeautifulSoup(page_pays, 'html.parser')

In [31]:
pays = soup.findAll(name = 'td', attrs = {'class': 'nowrap'})

pays = soup.select('tr:nth-child(11) .infobox-data')
    
gov_pays = [] 

for element in pays:
    gov_pays.append(element.text)
    
    
print('Government :',pays)



Government : [<td class="infobox-data"><span class="nowrap"><a href="/wiki/Unitary_state" title="Unitary state">Unitary</a> <a href="/wiki/Parliamentary_system" title="Parliamentary system">parliamentary</a><br/><a href="/wiki/Republic" title="Republic">constitutional republic</a></span></td>]


In [41]:
def wiki_search(url) : 

    url_open = urlopen("https://en.wikipedia.org/wiki/"+Pays_select)
    
    soup = BeautifulSoup(url_open, 'html.parser')
    
    details = soup('table',{'class':'infobox'})

    for i in details :
        h = i.find_all('tr')
        
        for j in h : 
            heading = j.find_all('th')
            detail = j.find_all('td')
            if heading is not None and detail is not None :
                for x,y in zip(heading,detail) :
                    print("{} : {}".format(x.text, y.text))
                    print("--------------------")
    
    for i in range(1,3) : 
        print(soup('p')[i].text)

wiki_search(url)


Capitaland largest city : Rome41°54′N 12°29′E﻿ / ﻿41.900°N 12.483°E﻿ / 41.900; 12.483
--------------------
Official languages : Italiana
--------------------
Native languages : See main article
--------------------
Ethnic groups  (2017)[1] : 91.5% Italians8.5% Others
--------------------
Religion  (2020)[2] : 80.8% Christianity13.4% No religion4.9% Islam0.9% Others
--------------------
Demonym(s) : Italian
--------------------
Government : Unitary parliamentaryconstitutional republic
--------------------
• President  : Sergio Mattarella
--------------------
• Prime Minister  : Mario Draghi
--------------------
• President of the Senate  : Elisabetta Casellati
--------------------
• President of theChamber of Deputies  : Roberto Fico
--------------------
Legislature : Parliament
--------------------
• Upper house : Senate of the Republic
--------------------
• Lower house : Chamber of Deputies
--------------------
• Unification  : 17 March 1861
--------------------
• Republic  : 2 June 

In [42]:
#Mise en forme des noms de pays pour 
country_region['Country name_2'] = country_region['Country name'].str.replace(' ', '_')
countries = country_region['Country name_2'].unique()
countries

array(['Finland', 'Denmark', 'Switzerland', 'Iceland', 'Netherlands',
       'Norway', 'Sweden', 'Luxembourg', 'New_Zealand', 'Austria',
       'Australia', 'Israel', 'Germany', 'Canada', 'Ireland',
       'Costa_Rica', 'United_Kingdom', 'Czech_Republic', 'United_States',
       'Belgium', 'France', 'Bahrain', 'Malta',
       'Taiwan_Province_of_China', 'United_Arab_Emirates', 'Saudi_Arabia',
       'Spain', 'Italy', 'Slovenia', 'Guatemala', 'Uruguay', 'Singapore',
       'Kosovo', 'Slovakia', 'Brazil', 'Mexico', 'Jamaica', 'Lithuania',
       'Cyprus', 'Estonia', 'Panama', 'Uzbekistan', 'Chile', 'Poland',
       'Kazakhstan', 'Romania', 'Kuwait', 'Serbia', 'El_Salvador',
       'Mauritius', 'Latvia', 'Colombia', 'Hungary', 'Thailand',
       'Nicaragua', 'Japan', 'Argentina', 'Portugal', 'Honduras',
       'Croatia', 'Philippines', 'South_Korea', 'Peru',
       'Bosnia_and_Herzegovina', 'Moldova', 'Ecuador', 'Kyrgyzstan',
       'Greece', 'Bolivia', 'Mongolia', 'Paraguay', 'Montenegro

In [43]:
req = requests.get("https://en.wikipedia.org/wiki/"+Pays_select)
soup = BeautifulSoup(req.text, "html.parser")

#Titre Page Wikipedia du pays
nom_page_pays = soup.title.string

In [44]:
print(soup.tr.contents)

print(soup.th.contents)

[<th class="infobox-above adr" colspan="2"><div class="fn org country-name">Italian Republic</div><div class="ib-country-names"><i lang="it" title="Italian-language text">Repubblica Italiana</i>  <span class="languageicon" style="font-size:100%;font-weight:normal;">(<a href="/wiki/Italian_language" title="Italian language">Italian</a>)</span></div></th>]
[<div class="fn org country-name">Italian Republic</div>, <div class="ib-country-names"><i lang="it" title="Italian-language text">Repubblica Italiana</i>  <span class="languageicon" style="font-size:100%;font-weight:normal;">(<a href="/wiki/Italian_language" title="Italian language">Italian</a>)</span></div>]


In [45]:
pays = soup.findAll(name = 'td', attrs = {'class': 'nowrap'})

pays = soup.select('tr:nth-child(11) .infobox-data')

pays

[<td class="infobox-data"><span class="nowrap"><a href="/wiki/Unitary_state" title="Unitary state">Unitary</a> <a href="/wiki/Parliamentary_system" title="Parliamentary system">parliamentary</a><br/><a href="/wiki/Republic" title="Republic">constitutional republic</a></span></td>]

In [46]:
gov_pays = [] 

for element in pays:
    gov_pays.append(element.text)
    
print('Government :',gov_pays)

Government : ['Unitary parliamentaryconstitutional republic']


In [47]:
req = requests.get("https://en.wikipedia.org/wiki/"+Pays_select)
soup = BeautifulSoup(req.text, "html.parser")

#Titre Page Wikipedia du pays
nom_page_pays = soup.title.string

table_class = 'infobox ib-country vcard'
infos_pays_table = soup.find('table', attrs={'class' : table_class})

df = pd.read_html(str(infos_pays_table))

table_pays = pd.DataFrame(df[0])

table_pays.columns = ['Label', 'info']

infos_final = table_pays[(table_pays['Label'] == 'Government') | (table_pays['Label'] == 'Legislature')]

infos_final

,Label,info
9,Government,Unitary parliamentaryconstitutional republic
15,Legislature,Parliament


In [48]:
table_pays

,Label,info
0,Flag Emblem,Flag Emblem
1,"Anthem: Il Canto degli Italiani (Italian)""The ...","Anthem: Il Canto degli Italiani (Italian)""The ..."
2,Location of Italy (dark green)– in Europe (lig...,Location of Italy (dark green)– in Europe (lig...
3,Capitaland largest city,Rome41°54′N 12°29′E﻿ / ﻿41.900°N 12.483°E
4,Official languages,Italiana
5,Native languages,See main article
6,Ethnic groups (2017)[1],91.5% Italians8.5% Others
7,Religion (2020)[2],80.8% Christianity13.4% No religion4.9% Islam0...
8,Demonym(s),Italian
9,Government,Unitary parliamentaryconstitutional republic
